In [2]:
subscription_key = '9a3169f5102c4616a966b0eb37073441'

In [3]:
import urllib.parse, http.client, json

headers = {
    'Ocp-Apim-Subscription-Key':subscription_key,
    'Content-Type':'application/json'
}

body = {
    'url':'https://static.independent.co.uk/s3fs-public/thumbnails/image/2018/04/24/15/angela-merkel.jpg'
}

params = urllib.parse.urlencode({
    'returnFaceId':True,
    'returnFaceLandmarks':True,
    'returnFaceAttributes':'age,gender,makeup,smile,glasses,accessories,emotion'
})

url = 'southcentralus.api.cognitive.microsoft.com'

endpoint = '/face/v1.0/detect?%s'

try:
    conn = http.client.HTTPSConnection(url)
    conn.request('POST', endpoint % params, str(body), headers)
    response = conn.getresponse()
    jsonData = response.read()
    data = json.loads(jsonData)
    detect_face_id = data[0]['faceId']
    print(detect_face_id)
    print(json.dumps(data, sort_keys=True, indent=2))
    conn.close()
except Exception as ex:
    print(ex)

0


In [28]:
group_id = 'random_people'

group_body = {
    'name':'Random People'
}

try:
    conn = http.client.HTTPSConnection('southcentralus.api.cognitive.microsoft.com')
    conn.request('PUT', '/face/v1.0/persongroups/%s' % group_id, str(group_body), headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as ex:
    print(ex)

b'{"error":{"code":"PersonGroupExists","message":"Person group \'random_people\' already exists."}}'


In [33]:
person_body = {
    'name':'Elizabeth'
}

person_endpoint = '/face/v1.0/persongroups/{0}/persons'.format(group_id)

try:
    conn = http.client.HTTPSConnection('southcentralus.api.cognitive.microsoft.com')
    conn.request('POST', person_endpoint, str(person_body), headers)
    response = conn.getresponse()
    jsonData = response.read()
    data = json.loads(jsonData)
    print(data)
    conn.close()
except Exception as ex:
    print(ex)

{'personId': '30969c57-3c95-49bc-a259-edb87a0fe777'}


In [42]:
try:
    conn = http.client.HTTPSConnection('southcentralus.api.cognitive.microsoft.com')
    conn.request('GET', person_endpoint, "", headers)
    response = conn.getresponse()
    jsonData = response.read()
    person_data = json.loads(jsonData)
    print(person_data)
    conn.close()
except Exception as ex:
    print(ex)

[{'personId': '775386ff-f289-4437-8154-19633a84006c', 'persistedFaceIds': ['831ef8a7-d9c3-40d5-a053-dbabd343eb0f', '967c19e4-d132-41b4-88dd-fd8f32bd7671', 'fff4242b-17ea-4bee-bf75-5e27ee54de43'], 'name': 'Bill', 'userData': None}, {'personId': 'b07f3b93-f078-4693-a00f-83b17cdf5f53', 'persistedFaceIds': ['8975dc7c-e732-4d68-897e-dcb9d200bdd2', 'bad76363-fd56-4915-8fd7-ea5348130604', 'e99ea0f1-96ac-4563-946a-4f427649e91d'], 'name': 'Angela', 'userData': None}, {'personId': 'f9e5cf62-de93-417a-90bb-3ced82e276a0', 'persistedFaceIds': ['6e28411e-4642-462d-a05a-f23135e8d36d', 'a33d477f-da3d-4366-98d0-4685e11d4921', 'b630a085-14cd-440f-9fa5-0db525f677b3'], 'name': 'Elizabeth', 'userData': None}]


In [35]:
try:
    conn = http.client.HTTPSConnection('southcentralus.api.cognitive.microsoft.com')
    conn.request('DELETE', person_endpoint+"/30969c57-3c95-49bc-a259-edb87a0fe777", "", headers)
    response = conn.getresponse()
    jsonData = response.read()
    print(jsonData)
    conn.close()
except Exception as ex:
    print(ex)

b''


In [22]:
import os

person_group_id = 'random_people'

face_headers = {
    'Ocp-Apim-Subscription-Key':subscription_key,
    'Content-Type':'application/octet-stream'
}

for person in person_data:
    face_endpoint = '/face/v1.0/persongroups/{0}/persons/{1}/persistedFaces'.format(person_group_id, person['personId'])
    for file in os.listdir('People/'+person['name']):
        with open('People/{0}/{1}'.format(person['name'],file), 'rb') as face_body:
            conn = http.client.HTTPSConnection('southcentralus.api.cognitive.microsoft.com')
            conn.request('POST', face_endpoint, face_body, face_headers)
            response = conn.getresponse()
            jsonData = response.read()
            data = json.loads(jsonData)
            print(data)
            conn.close()

{'persistedFaceId': '967c19e4-d132-41b4-88dd-fd8f32bd7671'}
{'persistedFaceId': '831ef8a7-d9c3-40d5-a053-dbabd343eb0f'}
{'persistedFaceId': 'fff4242b-17ea-4bee-bf75-5e27ee54de43'}
{'persistedFaceId': 'bad76363-fd56-4915-8fd7-ea5348130604'}
{'persistedFaceId': 'e99ea0f1-96ac-4563-946a-4f427649e91d'}
{'persistedFaceId': '8975dc7c-e732-4d68-897e-dcb9d200bdd2'}
{'persistedFaceId': '6e28411e-4642-462d-a05a-f23135e8d36d'}
{'persistedFaceId': 'b630a085-14cd-440f-9fa5-0db525f677b3'}
{'persistedFaceId': 'a33d477f-da3d-4366-98d0-4685e11d4921'}


In [37]:
train_headers = {
    'Ocp-Apim-Subscription-Key':subscription_key
}
train_endpoint = '/face/v1.0/persongroups/{0}/train'.format(person_group_id)

try:
    conn = http.client.HTTPSConnection('southcentralus.api.cognitive.microsoft.com')
    conn.request('POST', train_endpoint, '', train_headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as ex:
    print(ex)

b''


In [46]:
identify_endpoint = '/face/v1.0/identify'

identify_headers = {
    'Ocp-Apim-Subscription-Key':subscription_key
}

identify_body = {
    'faceIds':[detect_face_id],
    'personGroupId':person_group_id
}

try:
    conn = http.client.HTTPSConnection('southcentralus.api.cognitive.microsoft.com')
    conn.request('POST', identify_endpoint, str(identify_body), identify_headers)
    response = conn.getresponse()
    jsonData = response.read()
    data = json.loads(jsonData)
    print(data)
    conn.close()
except Exception as ex:
    print(ex)

[{'faceId': '7937a3a2-acfd-4049-b6cf-bdcd317d4561', 'candidates': [{'personId': 'b07f3b93-f078-4693-a00f-83b17cdf5f53', 'confidence': 0.71093}]}]


In [47]:
print(person_data)

[{'personId': '775386ff-f289-4437-8154-19633a84006c', 'persistedFaceIds': ['831ef8a7-d9c3-40d5-a053-dbabd343eb0f', '967c19e4-d132-41b4-88dd-fd8f32bd7671', 'fff4242b-17ea-4bee-bf75-5e27ee54de43'], 'name': 'Bill', 'userData': None}, {'personId': 'b07f3b93-f078-4693-a00f-83b17cdf5f53', 'persistedFaceIds': ['8975dc7c-e732-4d68-897e-dcb9d200bdd2', 'bad76363-fd56-4915-8fd7-ea5348130604', 'e99ea0f1-96ac-4563-946a-4f427649e91d'], 'name': 'Angela', 'userData': None}, {'personId': 'f9e5cf62-de93-417a-90bb-3ced82e276a0', 'persistedFaceIds': ['6e28411e-4642-462d-a05a-f23135e8d36d', 'a33d477f-da3d-4366-98d0-4685e11d4921', 'b630a085-14cd-440f-9fa5-0db525f677b3'], 'name': 'Elizabeth', 'userData': None}]


In [48]:
identified_person = data[0]
found = False
for candidate in identified_person['candidates']:
    for person in person_data:
        if(person['personId'] == candidate['personId']):
            print('FOUND IT!')
            print('This is ' + person['name'])
            found = True
            
if(found == False):
    print('This person could not be identified')

FOUND IT!
This is Angela
